In [47]:
from flask import Flask, request, jsonify
from flask_cors import CORS
import requests
from bs4 import BeautifulSoup
import pandas as pd
import pandas as pd
import re  # Para lidar com expressões regulares
import string  # Para lidar com pontuações
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords  # Para stopwords (é necessário ter a biblioteca NLTK instalada)
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import json

link = "http://localhost:8000"
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"}
requisicao = requests.get(link, headers=headers)
site = BeautifulSoup(requisicao.text, "html.parser")
# profissao = site.findAll("h4")
sobre = site.findAll("pre")
# pessoa = site.findAll("h2")
profissao = site.findAll("h4", {"class": "profissao"})
codigo = site.findAll("h3", {"class": "codigo"})

sobre_text = [element.get_text() for element in sobre]
profissao_text = [element.get_text() for element in profissao]
codigo_text = [element.get_text() for element in codigo]

dados = {
    'Sobre': sobre_text,
    'Profissao': profissao_text,
    'Codigo': codigo_text
}

df = pd.DataFrame(dados)

df['Profissao'] = df['Profissao'].apply(lambda x: x.lower()) #Conversão para letras minúsculas

# DATA MINING PARA LIMPEZA DE TEXTO:
# df['Sobre'] = df['Sobre'].str.replace('\r', ' ').str.replace('\n', ' ')
# df['Sobre'] = df['Sobre'].apply(lambda x: re.sub(r'[^a-zA-Z\s]', '', x)) #Remoção de caracteres especiais e números
df['Sobre'] = df['Sobre'].apply(lambda x: x.lower()) #Conversão para letras minúsculas
df['Sobre'] = df['Sobre'].apply(lambda x: re.sub(r'[.,"\'-?:!;]', '', x))
df['Sobre'] = df['Sobre'].apply(lambda x: re.sub('\n', '', x))
df['Sobre'] = df['Sobre'].apply(lambda x: re.sub('\r', '', x))
df['Sobre'] = df['Sobre'].apply(lambda x: re.sub('\t', '', x))
df['Sobre'] = df['Sobre'].apply(lambda x: re.sub('•', '', x))
# stop_words = set(stopwords.words('portuguese')) #Pegar stopwords em portugues
# df['Sobre'] = df['Sobre'].apply(lambda x: ' '.join(word for word in x.split() if word not in stop_words)) #Remoção de stopwords
# df['Sobre'] = df['Sobre'].apply(lambda x: ' '.join(x.split())) #Remoção de espaços em branco extras:
# df['Sobre'] = df['Sobre'].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation))) #Remoção de pontuações

# TOKEZIZAÇÃO DO TEXTO
# df['Sobre'] = df['Sobre'].apply(lambda x: word_tokenize(x)) #realizar a tokenização dos textos

# VETORIZAÇÃO DO TEXTO, REPRESENTADO OS TEXTOS EM CARACTERES NUMÉRICOS
# USANDO O TF-IDF
# tfidf_vectorizer = TfidfVectorizer() #Cria um objeto TfidfVectorizer
# tfidf_matrix = tfidf_vectorizer.fit_transform(df['Sobre'].apply(lambda x: ' '.join(x))) #Ajusta e transforma os tokens em uma matriz TF-IDF
# df_tfidf = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out()) #Crie um novo DataFrame com as representações TF-IDF

teste = {
  "cargo": "Scrum Master",
  "conhecimentos": [
    "Metodologia Ágil",
    "Scrum",
    "Kanban",
    "Lean",
    "Desenvolvimento de Software",
    "Facilitação",
    "Mentoria",
    "Resolução de Conflitos",
    "Gestão de Projetos",
    "Comunicação Interpessoal"
  ],
  "habilidades": [
    "Liderança Servidora",
    "Facilitação de Reuniões",
    "Capacidade de Escuta Ativa",
    "Tomada de Decisão",
    "Adaptabilidade",
    "Gestão de Tempo",
    "Visão Estratégica",
    "Empatia"
  ],
  "atitudes": [
    "Foco no Time",
    "Promoção da Autonomia",
    "Comprometimento",
    "Transparência",
    "Resiliência",
    "Honestidade",
    "Responsabilidade",
    "Melhoria Contínua"
  ],
  "filtro": 25
}

def pontuar_curriculo(texto):
    conhecimentos = sum(1 for palavra in teste["conhecimentos"] if palavra.lower() in texto)
    habilidades = sum(1 for palavra in teste["habilidades"] if palavra.lower() in texto)
    atitudes = sum(1 for palavra in teste["atitudes"] if palavra.lower() in texto)
    total = conhecimentos + habilidades + atitudes
    cont_co = sum(1 for palavra in teste["conhecimentos"] if palavra.lower() in teste["conhecimentos"])
    cont_ha = sum(1 for palavra in teste["habilidades"] if palavra.lower() in teste["conhecimentos"])
    cont_at = sum(1 for palavra in teste["atitudes"] if palavra.lower() in teste["conhecimentos"])
    porcentagem = (total / (cont_co + cont_ha + cont_at)) * 100
    return conhecimentos, habilidades, atitudes, total

df_pontuado = df.copy()

df_pontuado[['Pont_Conhecimentos', 'Pont_Habilidades', 'Pont_Atitudes', 'Total_Pontuacao']] = df_pontuado['Sobre'].apply(lambda x: pd.Series(pontuar_curriculo(x)))

df_p_s = df_pontuado.loc[df['Profissao'] == "scrum master"]

df_p_s.head()

# def match_profissao(a):  
#     profissoes = df.loc[df['Profissao'] == a["cargo"].lower()]

#     # profissoes_estruturado = profissoes['Codigo']
#     # profissoes_lista = profissoes_estruturado.values.flatten().tolist()
#     # return jsonify(profissoes_lista)

# app = Flask(__name__)
# CORS(app)

# @app.route('/scraping', methods=['POST'])
# def ask_question():
#     try:
#         # data = request.data.decode('utf-8')
#         data = request.json
#         return match_profissao(data)

#     except Exception as e:
#         return jsonify({"error": str(e)})

# if __name__ == '__main__':
#     app.run(host='0.0.0.0', port=7000)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Manhã\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Manhã\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,Sobre,Profissao,Codigo,Pont_Conhecimentos,Pont_Habilidades,Pont_Atitudes,Total_Pontuacao
0,profissional atuando como scrum master...,scrum master,305.162,5,0,0,5
2,resumo profissional profissiona...,scrum master,581.111,2,1,1,4
11,experiência com levantamento d...,scrum master,383.891,2,0,1,3
12,objetivo scrum master prodemge ...,scrum master,258.550,3,1,0,4
13,resumo scrum master squad lead...,scrum master,581.094,5,0,0,5
